In [137]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import seaborn as sns
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader

# **Problem description**

Simple model $x = \theta + \epsilon \ \text{with}\ \epsilon\sim \mathcal{N}(0 ; \sigma^{2})$

In [138]:
sigma = 0.1 #sigma²=0.01
mu0 = 3
sigma0 = 2

sigma_T = torch.Tensor([sigma])
mu0_T = torch.Tensor([mu0])
sigma0_T = torch.Tensor([sigma0])

base_distribution = torch.distributions.Normal(torch.Tensor([0.0]), torch.Tensor([1.0]))

In [139]:
def generate_dataset(N):
    res = torch.Tensor( N, 2)
    thetas = torch.distributions.Normal(mu0_T, sigma0_T).sample((N, 1))
    epsilons = torch.distributions.Normal(torch.Tensor([0]), sigma_T).sample((N, 1))
    x = thetas + epsilons
    res[:,0] = x.squeeze()
    res[:,1] = thetas.squeeze()
    return res

In [140]:
D = generate_dataset(1000)
D

tensor([[2.7310, 2.7182],
        [2.0699, 2.0615],
        [2.5964, 2.4708],
        ...,
        [4.2726, 4.4063],
        [0.0995, 0.2185],
        [1.3500, 1.3047]])

In [180]:
class flow(nn.Module):
    
    def __init__(self,omega, beta, gamma, base):#TODO init auto si alpha, beta, gamma pas précisé
        
        super(flow, self).__init__()
        self.omega = nn.Parameter(torch.Tensor([omega]), requires_grad=True)
        self.beta = nn.Parameter(torch.Tensor([beta]), requires_grad= True)
        self.gamma = nn.Parameter(torch.Tensor([gamma]), requires_grad= True)
        self.base = base
        
    def forward(self,z, x):
        theta = torch.exp(self.omega)*z + (self.beta * x + self.gamma)
        return theta
    
    def loss(self, thetas, X):
        res = 0
        '''for i,x in enumerate(X):
            est_mean = self.beta*x + self.gamma
            est_var = torch.exp(self.omega)**2
            logprob = torch.distributions.Normal(torch.Tensor(est_mean), torch.Tensor(est_var)).log_prob(thetas[i])
            res += logprob
        '''    
        est_mean = self.beta*x + self.gamma
        est_std = torch.exp(self.omega)
        logprob = torch.distributions.Normal(torch.Tensor(est_mean), torch.Tensor(est_std)).log_prob(thetas)
        res += logprob.sum()
        return - res.mean()
        

f = flow(1.0, 1.0, 1.0, base_distribution)

In [181]:
n_iter = 10000
learning_rate = 1e-3
optimizer = torch.optim.Adam(f.parameters(), learning_rate)

x, thetas = D[:,0], D[:,1]
for i in range(n_iter):
    
    loss_train = f.loss(thetas,x)
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    
    if i%20==0:
        print(f"Iter = {i} and loss = {loss_train}")


Iter = 0 and loss = 1987.188720703125
Iter = 20 and loss = 1959.2271728515625
Iter = 40 and loss = 1931.7666015625
Iter = 60 and loss = 1904.93310546875
Iter = 80 and loss = 1878.802978515625
Iter = 100 and loss = 1853.42236328125
Iter = 120 and loss = 1828.8125
Iter = 140 and loss = 1804.9681396484375
Iter = 160 and loss = 1781.8583984375
Iter = 180 and loss = 1759.4261474609375
Iter = 200 and loss = 1737.5911865234375
Iter = 220 and loss = 1716.2593994140625
Iter = 240 and loss = 1695.32763671875
Iter = 260 and loss = 1674.6966552734375
Iter = 280 and loss = 1654.2767333984375
Iter = 300 and loss = 1633.993408203125
Iter = 320 and loss = 1613.7900390625
Iter = 340 and loss = 1593.626220703125
Iter = 360 and loss = 1573.474365234375
Iter = 380 and loss = 1553.316650390625
Iter = 400 and loss = 1533.14208984375
Iter = 420 and loss = 1512.94189453125
Iter = 440 and loss = 1492.71044921875
Iter = 460 and loss = 1472.4429931640625
Iter = 480 and loss = 1452.1358642578125
Iter = 500 and lo

In [182]:
Z = torch.distributions.Normal(torch.Tensor([0]), torch.Tensor([1])).sample((1000, ))

z_prime = f.forward(Z, x)

#sns.kdeplot(data= z_prime.detach().squeeze())

#for xi in x:
#    plt.axvline(xi, color='blue')


In [183]:
C = (   mu0/(sigma0**2)  ) * ( 1/sigma0**2 + 1/sigma**2  )**(-1)

B = ( 1/sigma**2 ) * (1/sigma**2 + 1/sigma0**2 )**(-1)

A = ( 1/sigma0**2 + 1/sigma**2)
borne = mu0**2 + (1 + (sigma0**2)/(sigma**2) ) * (A**2)
borne

4030084.062499998

In [184]:
print(f" C = {C} and gamma = {f.gamma.item()}")
print(f"B = {B} ansd beta = {f.beta.item()}")
print(f" Â = {borne} and alpha^2 = {torch.exp(f.omega).item()**2}")

 C = 0.007481296758104739 and gamma = 0.012515312992036343
B = 0.9975062344139651 ansd beta = 0.9961519241333008
 Â = 4030084.062499998 and alpha^2 = 0.010383064157700128


todo: Vérifier si juste, comparer avec code de zuko